In [ ]:
# Ejemplo 3:
# Modelo TROPN, obtención del arbol de alcanzabilidad factible en el tiempo,
# y obtención de ruta óptima para el modelo propuesto por Mejía et al. (2018)


In [1]:
import snakes.plugins
import tropn
snakes.plugins.load(tropn, "snakes.nets", "nets")
from nets import *
from tropn import *
from time import process_time

In [2]:
n = PetriNet('TROPN-Mejia-2019-4x4')
n.globals.declare("from tropn import process")

# length of problem, lenght x lenght
lenght = 4

# add places
nplaces = lenght + 2
k = 1
for i in range(nplaces):
    bound = 0 if i==0 else k
    n.add_place(Place('p'+str(i),[], bound=bound))

tnumber = 1  #for correlative transition
ntokens = 1
njobs = lenght
# add input transtitions from p0 for each token
for p in n.place():
    if p.name != 'p0' and p.name != 'p'+str(nplaces-1):
        for i in range(ntokens): 
            guard = '(x=="X' + str(i+1) + '")'
            tname = 't'+str(tnumber)
            tnumber+=1
            n.add_transition(Transition(tname, Expression(guard)))
            n.add_input('p0', tname, MultiArc([Variable('x')]))
            n.add_output(p.name, tname, MultiArc([Expression('process(x)')]))
           
# add output transtitions from p0 for each token
for p in n.place():
    if p.name != 'p0' and p.name != 'p'+str(nplaces-1):
        for i in range(ntokens):
            guard = '(x=="X' + str(i+1) + '_' + str(njobs) + '")'
            tname = 't'+str(tnumber)
            tnumber+=1
            n.add_transition(Transition(tname, Expression(guard)))
            n.add_input(p.name, tname, MultiArc([Variable('x')]))
            n.add_output('p0', tname, MultiArc([Expression('x')]))
            
# add process transitions
for pinput in n.place():
    if pinput.name != 'p0' and pinput.name != 'p'+str(nplaces-1):
        for poutput in n.place(): 
            if poutput.name != 'p0' and poutput.name != 'p'+str(nplaces-1) and poutput != pinput:
                guard = ""
                for i in range(ntokens): 
                    for j in range(njobs-1):
                        guard += 'x=="X' + str(i+1) + '_' + str(j+1) + '" or '
                guard = '(' + guard[:-4] + ') and dot=="dot"'
                tname = 't'+str(tnumber)
                tnumber+=1
                n.add_transition(Transition(tname, Expression(guard)))
                n.add_input(pinput.name, tname, MultiArc([Variable('x')]))
                n.add_output(poutput.name, tname, MultiArc([Expression('process(x)')]))
                
# for place modeling robot, add arc to each transition
robotplace = 'p'+str(nplaces)
n.add_place(Place(robotplace,[], bound=1))
for t in n.transition():
    n.add_input(robotplace, t.name, MultiArc([Variable("dot")]))
    n.add_output(robotplace, t.name, MultiArc([Expression("dot")]))  

In [3]:
### delay matrix for process and movement
#              (d[i,j])=                 (M[i,j])=
#             54 9  38 95               3 4 1 1
#             34 15 19 34               1 1 2 3
#             61 89 28 7                4 2 3 2
#             2  70 87 29               2 3 4 4

delays = {}
d = [[54, 9, 38, 95], [34, 15, 19, 34], [61, 89, 28, 7], [2, 70, 87, 29]]
M = [[3, 4, 1, 1], [1, 1, 2, 3], [4, 2, 3, 2], [2, 3, 4, 4]]
for i in range(ntokens):
    for j in range(njobs):
        token = 'X'+str(i+1) if p==0 else 'X'+str(i+1)+'_'+str(j+1)
        delay = {}
        for m in range(lenght+1):
            for k in range(lenght):
                if m == M[k][j]:
                    delay.update({'p'+str(m):d[k][j]})
        delays.update({token:delay})
n.delays = delays

n.delays

{'X1_1': {'p1': 34, 'p2': 2, 'p3': 54, 'p4': 61},
 'X1_2': {'p1': 15, 'p2': 89, 'p3': 70, 'p4': 9},
 'X1_3': {'p1': 38, 'p2': 19, 'p3': 28, 'p4': 87},
 'X1_4': {'p1': 95, 'p2': 7, 'p3': 34, 'p4': 29}}

In [4]:
### delay matrix for process and movement
#              (d[i,j])=                 (M[i,j])=
#             54 9  38 95               3 4 1 1
#             34 15 19 34               1 1 2 3
#             61 89 28 7                4 2 3 2
#             2  70 87 29               2 3 4 4

delays = {}
d = [[54, 9, 38, 95, 54, 9, 38, 95,38, 95], 
     [34, 15, 19, 34, 34, 15, 19, 34,38, 95], 
     [61, 89, 28, 7, 61, 89, 28, 7,38, 95], 
     [2, 70, 87, 29, 2, 70, 87, 29,38, 95],
     [54, 9, 38, 95, 54, 9, 38, 95,38, 95], 
     [34, 15, 19, 34, 34, 15, 19, 34,38, 95], 
     [61, 89, 28, 7, 61, 89, 28, 7,38, 95], 
     [2, 70, 87, 29, 2, 70, 87, 29,38, 95],
     [61, 89, 28, 7, 61, 89, 28, 7,38, 95], 
     [2, 70, 87, 29, 2, 70, 87, 29,38, 95]]

M = [[1,2,3,4,5,6,7,8,9,10],
     [2,3,4,5,6,7,8,9,10,1],
     [3,4,5,6,7,8,9,10,1,2],
     [4,5,6,7,8,9,10,1,2,3],
     [5,6,7,8,9,10,1,2,3,4],
     [6,7,8,9,10,1,2,3,4,5],
     [7,8,9,10,1,2,3,4,5,6],
     [8,9,10,1,2,3,4,5,6,7],
     [9,10,1,2,3,4,5,6,7,8],
     [10,1,2,3,4,5,6,7,8,9]]
for i in range(ntokens):
    for j in range(njobs):
        token = 'X'+str(i+1) if p==0 else 'X'+str(i+1)+'_'+str(j+1)
        delay = {}
        for m in range(lenght+1):
            for k in range(lenght):
                if m == M[k][j]:
                    delay.update({'p'+str(m):d[k][j]})
        delays.update({token:delay})
n.delays = delays

n.delays

{'X1_1': {'p1': 54, 'p2': 34, 'p3': 61, 'p4': 2},
 'X1_2': {'p2': 9, 'p3': 15, 'p4': 89},
 'X1_3': {'p3': 38, 'p4': 19},
 'X1_4': {'p4': 95}}

In [5]:
def full_marking(s):
    m = s.marking
    r = Marking({})
    w = []
    for key in s.waiting.keys():
        w.append({key+'_r':MultiSet(s.waiting[key]['token'])})
    for key in s.process.keys():
        w.append({key+'_r':MultiSet(s.process[key]['token'])})
    for i in w:
        r = r + Marking(i)
        
    return m + r

In [6]:
# El procedimiento general de simulación es el siguiente
time_start = process_time()
# 1. Definir el estado inicial a partir de un marcaje inicial, con
#    ningún proceso actualmente en ejecución
m = (Marking({'p0': MultiSet(["X1"]*3),
              robotplace: MultiSet(["dot"])
             }))
s0 = NetState(m,{},{})
set_state(n,s0)
# 2. Obtener el árbol de alcanzabilidad
markings, tree, optimal = time_feasible_reachability_tree(n,s0)
# ....................
time_elapsed = (process_time() - time_start)
print ("%5.3f secs" % (time_elapsed))
print (len(markings), "states found")
print (len(tree), "state transitions")

3.425 secs
193 states found
275 state transitions


In [7]:
len(optimal)

28

In [8]:
for step in optimal:
    #print (step[5])
    #print (step[0])
    #print (step[1])
    #print (step[2], step[3])
    print (step[5], step[1])

0 {p0={'X1', 'X1', 'X1'}, p6={'dot'}}
0 {p0={'X1', 'X1'}, p2={'processing'}, p2_r={'X1_1'}, p6={'dot'}}
0 {p0={'X1'}, p2={'processing'}, p2_r={'X1_1'}, p3={'processing'}, p3_r={'X1_1'}, p6={'dot'}}
0 {p2={'processing'}, p2_r={'X1_1'}, p3={'processing'}, p3_r={'X1_1'}, p4={'processing'}, p4_r={'X1_1'}, p6={'dot'}}
2 {p2={'processing'}, p2_r={'X1_1'}, p3={'processing'}, p3_r={'X1_1'}, p4={'X1_1'}, p6={'dot'}}
2 {p1={'processing'}, p1_r={'X1_2'}, p2={'processing'}, p2_r={'X1_1'}, p3={'processing'}, p3_r={'X1_1'}, p6={'dot'}}
3 {p1={'X1_2'}, p2={'processing'}, p2_r={'X1_1'}, p3={'processing'}, p3_r={'X1_1'}, p6={'dot'}}
3 {p2={'processing'}, p2_r={'X1_1'}, p3={'processing'}, p3_r={'X1_1'}, p4={'processing'}, p4_r={'X1_3'}, p6={'dot'}}
22 {p2={'processing'}, p2_r={'X1_1'}, p3={'processing'}, p3_r={'X1_1'}, p4={'X1_3'}, p6={'dot'}}
22 {p1={'processing'}, p1_r={'X1_4'}, p2={'processing'}, p2_r={'X1_1'}, p3={'processing'}, p3_r={'X1_1'}, p6={'dot'}}
23 {p1={'X1_4'}, p2={'processing'}, p2_r={'X